In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
from tqdm import tqdm_notebook

In [2]:
url = 'https://www.reclameaqui.com.br/'

In [3]:
def get_soup(url,xpath):
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    try:
        element = driver.find_element_by_xpath(xpath)
        html_content = element.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content,'html.parser')
        return soup
    except:
        pass

In [5]:
##Scraping all the categories from the website

data = {}
soup = get_soup(url+'/categoria','//*[@id="all-categories"]/div/div')
for category in soup.findAll('a',{'class':'hidden-xs'}):
    data[category.find('div').h2.contents[0]] = {'category_link':category.get('href'),'companies_links':{}}

In [10]:
#getting all companies from categories rankings

for k,v in tqdm_notebook(data.items()):
    content = get_soup(url+v['category_link'],'//*[@id="category-rankings"]')
    for c in content.findAll('div',{'class':'box-gray'}):
        data[k]['companies_links'].update({c.h3.text:[]})
        for link in c.findAll('a',{'class':'business-name ng-binding'}):
            data[k]['companies_links'][c.h3.text].append(link.get('href'))


In [12]:
#Save json
with open('./exports/company_urls', 'w') as f:
    json.dump(data,f)

In [53]:
#Save csv
consolidated_data = []
for category in data:
    for sub in data[category]['companies_links']:
        for url in data[category]['companies_links'][sub]:
            consolidated_data.append((category,sub,url))
df = pd.DataFrame(consolidated_data, columns=['category','sub_category','company_url'])
df.to_csv('./exports/consolidate_company_urls.csv',index=False)